In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import RobustScaler # 使用 RobustScaler 更好地处理异常值
from sklearn.model_selection import TimeSeriesSplit # 保持时序交叉验证
import warnings
import gc # 导入垃圾回收模块
import os # 用于文件操作

warnings.filterwarnings('ignore')

In [ ]:
# Global configurations (可以根据你的硬件和数据进行调整)
SEED = 42
WINDOW_SIZE = 20          # 特征计算和LSTM序列长度
NUM_EPOCHS = 50           # 训练轮数 (可以适当增加，配合早停)
BATCH_SIZE = 64           # 批处理大小 (如果内存不足，减小此值)
HIDDEN_SIZE = 128         # LSTM隐藏层大小
NUM_LAYERS = 2            # LSTM层数 (双向LSTM实际层数会翻倍)
DROPOUT = 0.3             # Dropout比例
LEARNING_RATE = 0.001
N_SPLITS_CV = 5           # 时间序列交叉验证折数
MAX_SEQUENCES_TRAIN = 300000 # 限制训练时生成的总序列数，防止OOM，请根据内存调整

In [ ]:

# 设置随机种子
def set_seed(seed=SEED):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# ======================== 数据处理部分 ========================

class StockDataProcessor:
    def __init__(self, window_size=WINDOW_SIZE):
        self.window_size = window_size

    def _calculate_single_stock_features(self, df_single_stock):
        """为单个股票的DataFrame计算特征（假设已按日期排序）"""
        features = pd.DataFrame(index=df_single_stock.index)

        # 确保 '收盘', '开盘', '最高', '最低', '成交量' 列存在且为数值类型
        required_cols = ['收盘', '开盘', '最高', '最低', '成交量']
        for col in required_cols:
            if col not in df_single_stock.columns:
                raise ValueError(f"Error in _calculate_single_stock_features: Column '{col}' not found.")
            # 尝试转换为数值，无法转换的设为NaN
            df_single_stock[col] = pd.to_numeric(df_single_stock[col], errors='coerce')


        # 基础价格特征
        features['returns'] = df_single_stock['收盘'].pct_change()
        features['log_returns'] = np.log(df_single_stock['收盘'] / df_single_stock['收盘'].shift(1).replace(0, np.nan))

        # 价格相关特征
        features['high_low_ratio'] = df_single_stock['最高'] / df_single_stock['最低'].replace(0, np.nan)
        features['close_open_ratio'] = df_single_stock['收盘'] / df_single_stock['开盘'].replace(0, np.nan)

        # 成交量比率 (与过去20日均量比)
        volume_ma20 = df_single_stock['成交量'].rolling(window=20, min_periods=1).mean()
        features['volume_ratio_ma20'] = df_single_stock['成交量'] / volume_ma20.replace(0, np.nan)

        # RSI (Relative Strength Index)
        delta = df_single_stock['收盘'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14, min_periods=1).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14, min_periods=1).mean()
        rs = gain / loss.replace(0, np.nan) # 避免除以0
        features['rsi'] = 100 - (100 / (1 + rs))
        features['rsi'] = features['rsi'].fillna(50) # RSI通常在0-100，NaN时可设为中值50

        # 移动平均线 (MA) 及其与当前收盘价比率
        for period in [5, 10, 20, 60]:
            ma = df_single_stock['收盘'].rolling(window=period, min_periods=1).mean()
            features[f'ma_{period}'] = ma
            features[f'ma_ratio_{period}'] = df_single_stock['收盘'] / ma.replace(0, np.nan)

        # 布林带 (Bollinger Bands)
        ma20_bb = df_single_stock['收盘'].rolling(window=20, min_periods=1).mean()
        std20_bb = df_single_stock['收盘'].rolling(window=20, min_periods=1).std().fillna(0) # std为0时避免NaN
        features['bb_upper'] = ma20_bb + 2 * std20_bb
        features['bb_lower'] = ma20_bb - 2 * std20_bb
        bb_diff = (features['bb_upper'] - features['bb_lower']).replace(0, np.nan) # 避免带宽为0时除以0
        features['bb_width'] = bb_diff / ma20_bb.replace(0, np.nan)
        features['bb_position'] = (df_single_stock['收盘'] - features['bb_lower']) / bb_diff

        # MACD (Moving Average Convergence Divergence)
        exp1 = df_single_stock['收盘'].ewm(span=12, adjust=False, min_periods=1).mean()
        exp2 = df_single_stock['收盘'].ewm(span=26, adjust=False, min_periods=1).mean()
        features['macd'] = exp1 - exp2
        features['macd_signal'] = features['macd'].ewm(span=9, adjust=False, min_periods=1).mean()
        features['macd_diff'] = features['macd'] - features['macd_signal']

        # OBV (On-Balance Volume)
        obv_values = (np.sign(df_single_stock['收盘'].diff().fillna(0)) * df_single_stock['成交量']).fillna(0).cumsum()
        features['obv'] = obv_values
        # 对OBV进行归一化或取其变化率可能更有用，这里用原始值先
        features['obv_roc'] = features['obv'].pct_change() # OBV的变化率

        # 历史波动率 (Historical Volatility) - 20日收益率标准差
        features['volatility_20d'] = df_single_stock['收盘'].pct_change().rolling(window=20, min_periods=1).std()

        # 价格在N日窗口内的位置 (Price Position in N-day window)
        price_min_20d = df_single_stock['最低'].rolling(window=20, min_periods=1).min()
        price_max_20d = df_single_stock['最高'].rolling(window=20, min_periods=1).max()
        price_range_20d = (price_max_20d - price_min_20d).replace(0, np.nan)
        features['price_position_20d'] = (df_single_stock['收盘'] - price_min_20d) / price_range_20d

        # 随机振荡器 (Stochastic Oscillator %K and %D)
        low_14 = df_single_stock['最低'].rolling(window=14, min_periods=1).min()
        high_14 = df_single_stock['最高'].rolling(window=14, min_periods=1).min() #笔误修正：应为.max()
        high_14 = df_single_stock['最高'].rolling(window=14, min_periods=1).max()
        stoch_k_numerator = df_single_stock['收盘'] - low_14
        stoch_k_denominator = (high_14 - low_14).replace(0, np.nan)
        features['stoch_k'] = 100 * (stoch_k_numerator / stoch_k_denominator)
        features['stoch_d'] = features['stoch_k'].rolling(window=3, min_periods=1).mean() # %D is 3-day SMA of %K

        # ROC (Rate of Change)
        for period in [10, 20]: # 10日和20日价格变化率
            features[f'roc_{period}'] = df_single_stock['收盘'].pct_change(periods=period) * 100

        # 其他有用的基础特征
        if '换手率' in df_single_stock.columns:
            features['turnover_rate'] = pd.to_numeric(df_single_stock['换手率'], errors='coerce')
        if '振幅' in df_single_stock.columns:
            features['amplitude'] = pd.to_numeric(df_single_stock['振幅'], errors='coerce')


        features = features.replace([np.inf, -np.inf], np.nan) # 替换无穷值
        return features


In [ ]:
     def calculate_all_features(self, df_all_stocks):
        """对所有股票应用特征计算（按股票分组并排序）"""
        # 确保原始索引在排序和分组后能够恢复
        original_index = df_all_stocks.index
        # 按股票代码和日期排序，确保特征计算的正确性
        df_all_stocks_sorted = df_all_stocks.sort_values(['股票代码', '日期'])

        all_features_list = []
        for stock_code, group_df in df_all_stocks_sorted.groupby('股票代码'):
            # 对每个股票的副本进行操作，避免SettingWithCopyWarning
            single_stock_features = self._calculate_single_stock_features(group_df.copy())
            all_features_list.append(single_stock_features)

        if not all_features_list: # 如果没有任何股票数据或分组失败
            if not df_all_stocks.empty: # 如果原始df不为空，尝试获取列名
                 # 创建一个空的小DataFrame来获取预期的特征列名
                sample_cols_df = self._calculate_single_stock_features(df_all_stocks.iloc[:1].copy()) # 用第一行尝试
                return pd.DataFrame(columns=sample_cols_df.columns, index=original_index) # 返回带正确列和索引的空DF
            return pd.DataFrame(index=original_index) # 如果原始df也为空，返回完全空的DF

        final_features_df = pd.concat(all_features_list)
        final_features_df = final_features_df.reindex(original_index) # 恢复原始索引顺序
        return final_features_df

In [ ]:
def prepare_sequences(self, stock_data_with_raw_prices, features_df, max_sequences_limit=None):
        """
        准备序列数据用于LSTM。
        stock_data_with_raw_prices: 包含原始'日期', '股票代码', '收盘'列的DataFrame。
        features_df: 已计算好的特征DataFrame，索引与stock_data_with_raw_prices对应。
        max_sequences_limit: 限制生成的总序列数，用于内存控制。
        """
        X_sequences, y_targets, sequence_dates, sequence_stock_codes = [], [], [], []

        num_expected_features = features_df.shape[1]
        if num_expected_features == 0:
            print("警告: 特征数量为0。序列将为空。")
            return np.array([]).reshape(0, self.window_size, 0), np.array([]), [], []

        generated_sequence_count = 0

        # 按股票代码分组处理，确保每个股票的时间序列是连续的
        # 确保数据已按股票代码和日期排序
        # stock_data_with_raw_prices 和 features_df 应该有相同的索引并且已经对齐
        merged_df = stock_data_with_raw_prices[['日期', '股票代码', '收盘']].join(features_df)
        merged_df.sort_values(['股票代码', '日期'], inplace=True)

        for stock_code, group in merged_df.groupby('股票代码'):
            if max_sequences_limit and generated_sequence_count >= max_sequences_limit:
                break

            # 计算未来收益率作为目标
            group['future_return'] = group['收盘'].shift(-1) / group['收盘'] - 1
            group['future_return'] = group['future_return'].replace([np.inf, -np.inf], np.nan) # 处理极端值

            feature_columns_to_use = features_df.columns.tolist() # 这些是已经计算好的技术指标等

            # 从 window_size - 1 开始，因为我们需要 window_size 天的数据来形成一个序列
            # 到 len(group) - 1 结束，因为最后一个数据点没有未来收益率
            for i in range(self.window_size - 1, len(group) - 1):
                if max_sequences_limit and generated_sequence_count >= max_sequences_limit:
                    break

                target_return = group['future_return'].iloc[i]
                if pd.isna(target_return): # 如果未来收益率是NaN（通常是最后一天），则跳过
                    continue

                # 序列的起始和结束索引
                # 特征序列对应于 [i - (window_size - 1), i] 的数据
                # 目标是基于 i 时刻的数据预测 i+1 时刻的收益率
                sequence_start_idx = i - (self.window_size - 1)
                sequence_end_idx = i + 1 # 切片时不包含末尾，所以是 i+1

                # 提取特征序列
                current_feature_sequence = group[feature_columns_to_use].iloc[sequence_start_idx:sequence_end_idx].values

                # 确保序列长度正确
                if current_feature_sequence.shape[0] != self.window_size:
                    continue # 如果由于数据不足导致序列长度不够，则跳过

                X_sequences.append(current_feature_sequence.astype(np.float32))
                y_targets.append(target_return.astype(np.float32))
                sequence_dates.append(group['日期'].iloc[i])
                sequence_stock_codes.append(stock_code)
                generated_sequence_count += 1

        if not X_sequences: # 如果没有生成任何序列
             return np.array([]).reshape(0, self.window_size, num_expected_features), np.array([]), [], []

        return np.array(X_sequences), np.array(y_targets), sequence_dates, sequence_stock_codes

In [ ]:
# ======================== 模型部分 ========================
class AttentionLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS, dropout=DROPOUT):
        super(AttentionLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0, # LSTM自带的dropout只在多层时有效
            bidirectional=True # 使用双向LSTM
        )
        # 注意力机制的线性层
        # 输入维度是 hidden_size * 2 (因为是双向LSTM)
        self.attention_layer = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1)
        )
        # 全连接输出层
        self.fc_output = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size), # 输入仍然是双向LSTM的拼接输出
            nn.ReLU(),
            nn.Dropout(dropout), # 在FC层前加Dropout
            nn.Linear(hidden_size, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 1) # 输出一个值 (预测的收益率)
        )

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size)
        lstm_out, _ = self.lstm(x) # lstm_out shape: (batch_size, seq_len, hidden_size * 2)

        # 计算注意力权重
        # attention_layer的输入是 (batch_size, seq_len, hidden_size * 2)
        # attention_layer的输出是 (batch_size, seq_len, 1)
        attention_weights = torch.softmax(self.attention_layer(lstm_out), dim=1) # 在时间步维度上softmax

        # 应用注意力权重
        # context_vector shape: (batch_size, hidden_size * 2)
        context_vector = torch.sum(attention_weights * lstm_out, dim=1) # 加权求和

        # 通过全连接层输出
        output = self.fc_output(context_vector) # output shape: (batch_size, 1)
        return output

In [ ]:
class StockDataset(Dataset):
    def __init__(self, X_data, y_data=None):
        # 确保数据是 float32 类型的 NumPy 数组
        self.X_data = X_data.astype(np.float32) if isinstance(X_data, np.ndarray) else np.array(X_data, dtype=np.float32)
        if y_data is not None:
            self.y_data = y_data.astype(np.float32) if isinstance(y_data, np.ndarray) else np.array(y_data, dtype=np.float32)
        else:
            self.y_data = None

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, idx):
        if self.y_data is not None:
            # 直接返回tensor，避免在训练循环中转换
            return torch.from_numpy(self.X_data[idx]), torch.tensor(self.y_data[idx])
        else:
            return torch.from_numpy(self.X_data[idx])

In [ ]:
# ======================== 训练和预测封装类 ========================
class StockPredictor:
    def __init__(self, window_size=WINDOW_SIZE, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS, dropout=DROPOUT):
        self.window_size = window_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.device = device
        self.trained_models_states = [] # 存储每个fold训练好的模型state_dict
        self.trained_scalers = []     # 存储每个fold对应的scaler
        self.trained_feature_columns = None # 存储训练时使用的特征列名

    def train(self, train_data_df, num_epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, n_cv_splits=N_SPLITS_CV, max_sequences_limit=MAX_SEQUENCES_TRAIN):
        processor = StockDataProcessor(self.window_size)

        print("步骤1: 计算技术指标特征...")
        # 传入原始的train_data_df用于特征计算
        features_df = processor.calculate_all_features(train_data_df.copy()) # 使用副本避免修改原始数据

        # 特征后处理：填充NaN值
        # 优先使用前向填充，然后后向填充，最后用0填充剩余的（通常是开始几行）
        features_df = features_df.ffill().bfill().fillna(0)
        features_df = features_df.replace([np.inf, -np.inf], 0) # 再次确保无穷值被处理

        if features_df.empty:
            raise ValueError("特征计算结果为空，无法继续训练。")

        self.trained_feature_columns = features_df.columns.tolist() # 保存特征列名
        print(f"计算得到的特征数量: {len(self.trained_feature_columns)}")

        print("步骤2: 准备序列数据...")
        # 传入包含原始价格的train_data_df和计算好的features_df
        X_all_sequences, y_all_targets, _, _ = processor.prepare_sequences(
            train_data_df, features_df, max_sequences_limit=max_sequences_limit
        )

        del features_df # 释放内存
        gc.collect()

        if X_all_sequences.size == 0 or len(X_all_sequences) == 0:
            raise ValueError("序列数据为空，无法训练。请检查数据和prepare_sequences逻辑。")

        # 最终检查序列中的NaN/Inf，并用0替换
        if np.isnan(X_all_sequences).any() or np.isinf(X_all_sequences).any():
            print("警告: 序列X中发现NaN/Inf，进行最终清理...")
            X_all_sequences = np.nan_to_num(X_all_sequences, nan=0.0, posinf=0.0, neginf=0.0)
        if np.isnan(y_all_targets).any() or np.isinf(y_all_targets).any():
            print("警告: 目标y中发现NaN/Inf，进行最终清理...")
            y_all_targets = np.nan_to_num(y_all_targets, nan=0.0, posinf=0.0, neginf=0.0)

        print(f"总共生成序列数量: {len(X_all_sequences)}, 每条序列长度: {X_all_sequences.shape[1]}, 特征维度: {X_all_sequences.shape[2]}")

        # 使用TimeSeriesSplit进行交叉验证
        time_series_cv = TimeSeriesSplit(n_splits=n_cv_splits)

        self.trained_models_states = [] # 清空旧模型
        self.trained_scalers = []     # 清空旧scaler

        for fold_num, (train_indices, val_indices) in enumerate(time_series_cv.split(X_all_sequences)):
            print(f"\n--- 开始训练第 {fold_num + 1}/{n_cv_splits} 折 ---")
            X_train_fold, X_val_fold = X_all_sequences[train_indices], X_all_sequences[val_indices]
            y_train_fold, y_val_fold = y_all_targets[train_indices], y_all_targets[val_indices]

            # 数据缩放：对当前训练折的数据拟合RobustScaler，并应用于训练集和验证集
            # RobustScaler对每个特征独立操作
            # X_train_fold shape: (num_samples, sequence_length, num_features)
            # 需要reshape成2D进行缩放: (num_samples * sequence_length, num_features)
            current_fold_scaler = RobustScaler()
            num_features = X_train_fold.shape[-1]

            # 拟合scaler
            X_train_fold_reshaped = X_train_fold.reshape(-1, num_features)
            current_fold_scaler.fit(X_train_fold_reshaped) # 只在训练数据上fit

            # 应用scaler
            X_train_fold_scaled_reshaped = current_fold_scaler.transform(X_train_fold_reshaped)
            X_train_fold_scaled = X_train_fold_scaled_reshaped.reshape(X_train_fold.shape)

            X_val_fold_reshaped = X_val_fold.reshape(-1, num_features)
            X_val_fold_scaled_reshaped = current_fold_scaler.transform(X_val_fold_reshaped)
            X_val_fold_scaled = X_val_fold_scaled_reshaped.reshape(X_val_fold.shape)

            self.trained_scalers.append(current_fold_scaler) # 保存这个fold的scaler

            # 创建Dataset和DataLoader
            train_fold_dataset = StockDataset(X_train_fold_scaled, y_train_fold)
            val_fold_dataset = StockDataset(X_val_fold_scaled, y_val_fold)

            # pin_memory=True 可以在GPU训练时加速数据转移，num_workers=0 在主进程加载数据
            train_fold_loader = DataLoader(train_fold_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
            val_fold_loader = DataLoader(val_fold_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

            # 初始化模型
            model_for_fold = AttentionLSTM(
                input_size=num_features,
                hidden_size=self.hidden_size,
                num_layers=self.num_layers,
                dropout=self.dropout
            ).to(self.device)

            criterion = nn.MSELoss() # 均方误差损失
            optimizer = optim.Adam(model_for_fold.parameters(), lr=LEARNING_RATE)
            # ReduceLROnPlateau: 当验证损失不再改善时降低学习率
            lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

            best_validation_loss = float('inf')
            early_stopping_patience = 10 # 如果连续10个epoch验证损失没有改善，则早停
            patience_count = 0
            best_model_state_for_fold = None

            for epoch in range(num_epochs):
                model_for_fold.train() # 设置为训练模式
                total_train_loss_epoch = 0
                for batch_X, batch_y in train_fold_loader:
                    batch_X, batch_y = batch_X.to(self.device), batch_y.to(self.device).unsqueeze(1) # 目标y也需要是 (batch, 1)

                    optimizer.zero_grad()
                    predictions = model_for_fold(batch_X)
                    loss = criterion(predictions, batch_y)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model_for_fold.parameters(), max_norm=1.0) # 梯度裁剪防止爆炸
                    optimizer.step()
                    total_train_loss_epoch += loss.item()

                avg_train_loss_epoch = total_train_loss_epoch / len(train_fold_loader)

                # 验证步骤
                model_for_fold.eval() # 设置为评估模式
                total_val_loss_epoch = 0
                with torch.no_grad():
                    for batch_X_val, batch_y_val in val_fold_loader:
                        batch_X_val, batch_y_val = batch_X_val.to(self.device), batch_y_val.to(self.device).unsqueeze(1)
                        val_predictions = model_for_fold(batch_X_val)
                        val_loss = criterion(val_predictions, batch_y_val)
                        total_val_loss_epoch += val_loss.item()

                avg_val_loss_epoch = total_val_loss_epoch / len(val_fold_loader)
                lr_scheduler.step(avg_val_loss_epoch) # 根据验证损失调整学习率

                if (epoch + 1) % 5 == 0 or epoch == num_epochs - 1 : # 每5轮或最后一轮打印
                    print(f"Fold {fold_num+1}, Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss_epoch:.6f}, Val Loss: {avg_val_loss_epoch:.6f}, LR: {optimizer.param_groups[0]['lr']:.7f}")

                if avg_val_loss_epoch < best_validation_loss:
                    best_validation_loss = avg_val_loss_epoch
                    best_model_state_for_fold = model_for_fold.state_dict() # 保存最佳模型的状态
                    patience_count = 0
                else:
                    patience_count += 1

                if patience_count >= early_stopping_patience:
                    print(f"Fold {fold_num+1}: Early stopping at epoch {epoch+1} due to no improvement in validation loss.")
                    break

                gc.collect()
                if torch.cuda.is_available(): torch.cuda.empty_cache()

            if best_model_state_for_fold:
                self.trained_models_states.append(best_model_state_for_fold)
            else: # 如果由于某种原因（例如训练过早结束且从未改善）没有最佳状态，则保存最后一个状态
                print(f"Warning: Fold {fold_num+1} did not find a best model state based on validation loss improvement. Saving last state.")
                self.trained_models_states.append(model_for_fold.state_dict())

            del model_for_fold, train_fold_loader, val_fold_loader, train_fold_dataset, val_fold_dataset # 清理内存
            del X_train_fold_scaled, X_val_fold_scaled, X_train_fold_reshaped, X_val_fold_reshaped
            gc.collect()
            if torch.cuda.is_available(): torch.cuda.empty_cache()

        del X_all_sequences, y_all_targets # 清理大块数据
        gc.collect()
        print("\n--- 所有折训练完成 ---")


    def predict(self, test_data_df):
        if not self.trained_models_states or not self.trained_scalers or not self.trained_feature_columns:
            print("错误: 模型未训练或scaler/特征列信息丢失。无法预测。")
            return self._create_empty_submission()

        processor = StockDataProcessor(self.window_size)
        print("\n步骤1 (预测): 为测试数据计算特征...")
        test_features_df = processor.calculate_all_features(test_data_df.copy())

        # 使用训练时确定的特征列，并按相同顺序排列
        # 对于测试集中可能由于数据不足而无法计算的特征，需要填充
        # 同时处理训练时存在但测试特征计算后可能丢失的列（例如全为NaN被drop）
        aligned_test_features_df = pd.DataFrame(columns=self.trained_feature_columns, index=test_features_df.index)
        for col in self.trained_feature_columns:
            if col in test_features_df.columns:
                aligned_test_features_df[col] = test_features_df[col]

        aligned_test_features_df = aligned_test_features_df.ffill().bfill().fillna(0) # 统一填充
        aligned_test_features_df = aligned_test_features_df.replace([np.inf, -np.inf], 0)

        print("步骤2 (预测): 准备预测序列...")
        # 我们只需要每个股票最后 window_size 天的特征数据来做预测
        # 注意：这里不计算目标变量 (future_return)

        # 获取每个股票在测试数据中最后可用的日期
        # last_available_date_per_stock = test_data_df.groupby('股票代码')['日期'].max()
        # test_data_df_for_pred 是为了获取股票代码和日期，用于从 aligned_test_features_df 中提取最后的序列

        X_predict_list = []
        predict_stock_codes_order = [] # 存储预测序列对应的股票代码顺序

        # 确保测试数据也按股票代码和日期排序，以便提取最后的序列
        test_data_df_sorted = test_data_df.sort_values(['股票代码', '日期'])

        for stock_code, group_df in test_data_df_sorted.groupby('股票代码'):
            if len(group_df) < self.window_size: # 如果数据不足以形成一个完整序列，则跳过
                # print(f"股票 {stock_code} 数据不足 {self.window_size} 天，跳过预测。")
                continue

            # 获取该股票对应的特征数据 (已经过对齐和填充)
            stock_features = aligned_test_features_df.loc[group_df.index]

            # 取最后的 window_size 条特征记录
            last_feature_sequence = stock_features.iloc[-self.window_size:].values

            # 再次检查NaN/Inf，以防万一
            if np.isnan(last_feature_sequence).any() or np.isinf(last_feature_sequence).any():
                # print(f"警告: 股票 {stock_code} 的最后序列中仍存在NaN/Inf，进行清理。")
                last_feature_sequence = np.nan_to_num(last_feature_sequence, nan=0.0, posinf=0.0, neginf=0.0)

            X_predict_list.append(last_feature_sequence.astype(np.float32))
            predict_stock_codes_order.append(stock_code)

        if not X_predict_list:
            print("错误: 未能为任何股票生成有效的预测输入序列。")
            return self._create_empty_submission()

        X_predict_np = np.array(X_predict_list)
        num_features = X_predict_np.shape[-1]

        # 集成预测：对每个fold训练的模型和scaler进行预测，然后平均
        ensemble_predictions_sum = np.zeros(len(predict_stock_codes_order))
        num_valid_models = 0

        print(f"步骤3 (预测): 使用 {len(self.trained_models_states)} 个已训练模型进行集成预测...")
        for model_idx, (model_state, scaler) in enumerate(zip(self.trained_models_states, self.trained_scalers)):
            # print(f"  使用模型 {model_idx+1}...")
            current_model = AttentionLSTM(
                input_size=num_features,
                hidden_size=self.hidden_size,
                num_layers=self.num_layers,
                dropout=self.dropout
            ).to(self.device)
            current_model.load_state_dict(model_state)
            current_model.eval()

            # 使用对应的scaler对当前批次的预测数据进行缩放
            # X_predict_np shape: (num_stocks, window_size, num_features)
            # scaler expects 2D: (num_stocks * window_size, num_features)
            X_predict_reshaped = X_predict_np.reshape(-1, num_features)
            X_predict_scaled_reshaped = scaler.transform(X_predict_reshaped)
            X_predict_scaled_for_model = X_predict_scaled_reshaped.reshape(X_predict_np.shape)

            X_predict_tensor = torch.from_numpy(X_predict_scaled_for_model).to(self.device)

            with torch.no_grad():
                fold_predictions = current_model(X_predict_tensor).cpu().numpy().squeeze() # squeeze to get (num_stocks,)

            # 确保fold_predictions是一维的，如果只有一个股票预测，可能需要调整
            if fold_predictions.ndim == 0: # 如果只有一个预测结果，它可能是标量
                fold_predictions = np.array([fold_predictions])

            if len(fold_predictions) == len(ensemble_predictions_sum):
                 ensemble_predictions_sum += fold_predictions
                 num_valid_models +=1
            else:
                print(f"警告: 模型 {model_idx+1} 的预测数量 ({len(fold_predictions)}) 与预期 ({len(ensemble_predictions_sum)}) 不符，已跳过此模型的预测。")

            del current_model, X_predict_tensor # 清理内存
            gc.collect()
            if torch.cuda.is_available(): torch.cuda.empty_cache()

        if num_valid_models == 0:
            print("错误：没有一个模型成功做出预测。")
            return self._create_empty_submission()

        final_avg_predictions = ensemble_predictions_sum / num_valid_models

        # 构建结果DataFrame
        results_list = []
        for stock_code, predicted_return in zip(predict_stock_codes_order, final_avg_predictions):
            # 确保收益率是有效的数值
            if pd.notna(predicted_return) and np.isfinite(predicted_return):
                 results_list.append({'StockCode': stock_code, 'PredictedReturn': predicted_return})
            else:
                 results_list.append({'StockCode': stock_code, 'PredictedReturn': 0.0}) # 对无效预测赋0


        predictions_summary_df = pd.DataFrame(results_list)
        predictions_summary_df.sort_values(by='PredictedReturn', ascending=False, inplace=True)

        # print("\n预测收益率汇总 (前5和后5):")
        # print(pd.concat([predictions_summary_df.head(), predictions_summary_df.tail()]))


        top_10_stocks = predictions_summary_df['StockCode'].head(10).tolist()
        # 对于涨幅最小的，需要反向排序或取尾部。如果直接取tail，已经是升序（因为前面是降序）
        # 我们需要的是 "PredictedReturn" 最小的10个
        bottom_10_stocks_df = predictions_summary_df.sort_values(by='PredictedReturn', ascending=True)
        bottom_10_stocks = bottom_10_stocks_df['StockCode'].head(10).tolist()


        # 确保输出列表长度为10，不足则用占位符（或空字符串，根据比赛要求）
        placeholder = '' # 使用空字符串作为占位符
        while len(top_10_stocks) < 10: top_10_stocks.append(placeholder)
        while len(bottom_10_stocks) < 10: bottom_10_stocks.append(placeholder)

        submission_df = pd.DataFrame({
            '涨幅最大股票代码': top_10_stocks,
            '涨幅最小股票代码': bottom_10_stocks
        })
        return submission_df

    def _create_empty_submission(self):
        placeholder = '' # 确保占位符一致
        return pd.DataFrame({
            '涨幅最大股票代码': [placeholder] * 10,
            '涨幅最小股票代码': [placeholder] * 10
        })

In [ ]:
# ======================== 主程序 ========================
def main():
    print("=== 开始股票预测模型训练与预测 (优化版) ===")

    try:
        print("1. 读取数据...")
        train_df_raw = pd.read_csv('/kaggle/input/2025bdc/train.csv', encoding='utf-8')
        test_df_raw = pd.read_csv('/kaggle/input/2025bdc/test.csv', encoding='utf-8')
        print(f"  训练数据 {train_df_raw.shape}, 测试数据 {test_df_raw.shape}")

        # 关键列名检查与预处理 (挪到 StockDataProcessor 或 predictor.train/predict 内部更合适，这里简化)
        # 这里仅作示例，实际应在数据处理类中更细致地完成
        for df in [train_df_raw, test_df_raw]:
            df['日期'] = pd.to_datetime(df['日期'])
            # 确保股票代码是字符串类型，以便正确分组
            if '股票代码' in df.columns:
                 df['股票代码'] = df['股票代码'].astype(str).str.zfill(6) # 格式化为6位字符串，如 '000001'
            # 检查并转换数值列，这里只列举几个核心的，StockDataProcessor._calculate_single_stock_features 会处理更多
            cols_to_numeric = ['开盘', '收盘', '最高', '最低', '成交量']
            for col in cols_to_numeric:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            # 必须的列如果转换后大量NaN，可能需要更复杂的填充或数据清洗策略
            df.dropna(subset=cols_to_numeric, inplace=True) # 删除核心价格数据无效的行


        if train_df_raw.empty:
            print("错误: 训练数据为空或预处理后为空，无法继续。")
            return

        print("\n2. 初始化并训练模型...")
        stock_predictor = StockPredictor(
            window_size=WINDOW_SIZE,
            hidden_size=HIDDEN_SIZE,
            num_layers=NUM_LAYERS,
            dropout=DROPOUT
        )
        stock_predictor.train(
            train_df_raw,
            num_epochs=NUM_EPOCHS,
            batch_size=BATCH_SIZE,
            n_cv_splits=N_SPLITS_CV,
            max_sequences_limit=MAX_SEQUENCES_TRAIN # 全局配置的限制
        )

        print("\n3. 生成预测结果...")
        if test_df_raw.empty:
            print("警告: 测试数据为空，将生成空结果文件。")
            final_submission_df = stock_predictor._create_empty_submission()
        else:
            final_submission_df = stock_predictor.predict(test_df_raw)

        print("\n4. 保存结果到 result.csv...")
        final_submission_df.to_csv('result1.csv', index=False, encoding='utf-8')
        print("  结果已保存。输出样本:")
        print(final_submission_df.head())

    except Exception as e:
        print(f"!!! 程序执行过程中发生严重错误: {e}")
        import traceback
        traceback.print_exc()
        # 尝试生成一个空的提交文件，以满足比赛提交流程
        if 'stock_predictor' in locals() and isinstance(stock_predictor, StockPredictor):
            final_submission_df = stock_predictor._create_empty_submission()
        else: # 如果predictor都未初始化
            placeholder = ''
            final_submission_df = pd.DataFrame({
                '涨幅最大股票代码': [placeholder] * 10,
                '涨幅最小股票代码': [placeholder] * 10
            })
        final_submission_df.to_csv('result.csv', index=False, encoding='utf-8')
        print("  由于发生错误，已生成一个空的或占位的result.csv。")

    finally:
        print("\n清理资源...")
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        print("=== 程序执行完毕 ===")

if __name__ == "__main__":
    main()